In [1]:
import os
import sys

local_path = os.getcwd()
# 设置工作目录为项目的主目录
os.chdir(os.path.join(local_path, "../../"))  # 使用相对路径将工作目录切换到 project 文件夹
print("Current working directory:", os.getcwd())
project_path = os.path.abspath(os.path.join(local_path, "../../"))
sys.path.append(project_path)   #将模块查找路径切换

save_path = os.path.join(project_path, "result/analyse_SIL/")
print(save_path)

Current working directory: /home/yixiong_ren/work/FBM
/home/yixiong_ren/work/FBM/result/analyse_SIL/


In [2]:
import torch
# 检查是否有可用的GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


# 设置随机种子

In [17]:
import numpy as np
import random

# 使用示例
utils.set_seed(42)  # 42 是一个示例种子数，您可以根据需求更改

# 加载数据集

In [4]:
import utils 

batch_size = 64

train_loader = utils.get_dataloader("MNIST", batch_size=batch_size, train=True, class_counts=[100]*10)
test_loader = utils.get_dataloader("MNIST", batch_size=batch_size, train=False)

In [5]:
# 获取 DataLoader 中的全部数据
train_features = []
train_labels = []

for batch_data, batch_labels in train_loader:
    train_features.append(batch_data.view(-1, 784))
    train_labels.append(batch_labels)

# 合并所有批次
train_features = torch.cat(train_features, dim=0)
train_labels = torch.cat(train_labels, dim=0)

# 获取 DataLoader 中的全部数据
test_features = []
test_labels = []

for batch_data, batch_labels in train_loader:
    test_features.append(batch_data.view(-1, 784))
    test_labels.append(batch_labels)

# 合并所有批次
test_features = torch.cat(test_features, dim=0)
test_labels = torch.cat(test_labels, dim=0)

## 绘图

In [6]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# 可视化函数：将数据降维到 2D 并可视化
def visualize_2d(features, labels, savepath=None):
    """
    使用 t-SNE 将高维数据降维到 2D 并可视化。
    :param features: 高维特征数据 (tensor or numpy array)
    :param labels: 标签 (tensor or numpy array)
    """
    # 转换为 numpy
    features_np = features.detach().numpy()
    labels_np = labels.numpy()

    # 使用 t-SNE 降维到 2D
    _, dims =features.shape
    
    if dims > 2:
        #tsne = TSNE(n_components=2, random_state=42, perplexity=40)
        pca = PCA(n_components=2)
        features_2d = pca.fit_transform(features_np)
        explained_variance_ratio = pca.explained_variance_ratio_
    else:
        features_2d = features_np
        explained_variance_ratio = None

    # 绘制 2D 散点图
    plt.figure(figsize=(8, 6))
    for label in np.unique(labels):
        mask = labels_np == label
        plt.scatter(
            features_2d[mask, 0],
            features_2d[mask, 1],
            label=f"Class {label}",
            alpha=0.6
        )
    plt.xlabel("Dimension 1")
    plt.ylabel("Dimension 2")
    plt.title("Visualization of High-Dimensional Data")
    plt.legend()
    plt.grid(True)

    # 显示 PCA 信息占比
    if explained_variance_ratio is not None:
        info_text = f"Explained Variance:\nDim 1: {explained_variance_ratio[0]:.2%}\nDim 2: {explained_variance_ratio[1]:.2%}"
        plt.text(0.05, 0.95, info_text, transform=plt.gca().transAxes, fontsize=10,
                 verticalalignment='top', bbox=dict(boxstyle='round', alpha=0.2))

    if savepath is None:
        plt.show()
    else:
        plt.savefig(savepath+'.png')
    plt.close()
    return None

# 训练

In [18]:
import torch.optim as optim
import torch.nn.functional as F
from loss.loss import FBMLoss
from models.modelset import FBMLayer

# 定义超参数
input_size = 784
hidden_dim = 1000
num_classes = 10      # MNIST有10个类别
learning_rate = 0.01
num_epochs = 100
batch_size = 64
b = 1.5
a = 2

# 实例化模型、定义损失函数和优化器
model = FBMLayer(input_size, hidden_dim).to(device)
criterion = FBMLoss(hidden_dim, 0.01, a, b, losstype="fast_StrongInter")
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

model.train()
# 训练模型
for epoch in range(num_epochs):
    for images, labels in train_loader:
        # 将图像和标签移动到 GPU 上
        images = images.view(-1, input_size).to(device)  # 展平图像并转移到 GPU
        labels = labels.to(device)  # 标签移动到 GPU
        #labels_one_hot = F.one_hot(labels, num_classes=num_classes).float()
        
        # 前向传播
        outputs = model(images)
        #loss = criterion(outputs, labels_one_hot, model.linear.weight)
        loss = criterion(outputs, labels, model.linear.weight)
        
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    '''分析代码'''
    with torch.no_grad():
        local_data = train_features.view(-1, input_size).to(device)
        analyse_data = model(local_data).cpu()
        
        local_save_path = os.path.join(save_path, f"output_{a:.2f}_{b:.2f}/")
        os.makedirs(local_save_path, exist_ok=True)

        visualize_2d(analyse_data, train_labels.cpu(), os.path.join(local_save_path, f"{epoch}"))   #画出最后的结果
        out_dis = utils.fast_FBDistance(analyse_data, train_labels.cpu())
        # 计算对角项的和
        diagonal_sum = torch.diag(out_dis).sum()

        # 计算非对角项的和
        total_sum = out_dis.sum()
        non_diagonal_sum = total_sum - diagonal_sum
        with open(os.path.join(local_save_path, "distance.txt"), "a") as file:
            file.write("="*20+"\n"+f"epoch={epoch}"+"\n")
            file.write(str(out_dis) + "\n")  # 写入数据并换行
            file.write(f"Bosen={diagonal_sum}\t Fermi={non_diagonal_sum}\n")
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/100], Loss: -15.9489
Epoch [2/100], Loss: -17.6740
Epoch [3/100], Loss: -18.5091
Epoch [4/100], Loss: -20.7987
Epoch [5/100], Loss: -16.2474
Epoch [6/100], Loss: -15.7654
Epoch [7/100], Loss: -17.0134
Epoch [8/100], Loss: -18.2751
Epoch [9/100], Loss: -20.2878
Epoch [10/100], Loss: -20.9477
Epoch [11/100], Loss: -18.0155
Epoch [12/100], Loss: -18.3732
Epoch [13/100], Loss: -19.3668
Epoch [14/100], Loss: -19.9940
Epoch [15/100], Loss: -18.3440
Epoch [16/100], Loss: -18.2764
Epoch [17/100], Loss: -16.0233
Epoch [18/100], Loss: -21.1292
Epoch [19/100], Loss: -16.9698
Epoch [20/100], Loss: -18.4256
Epoch [21/100], Loss: -20.0123
Epoch [22/100], Loss: -20.5775
Epoch [23/100], Loss: -16.1305
Epoch [24/100], Loss: -21.2787
Epoch [25/100], Loss: -20.5678
Epoch [26/100], Loss: -19.8668
Epoch [27/100], Loss: -19.1025
Epoch [28/100], Loss: -20.5015
Epoch [29/100], Loss: -17.4643
Epoch [30/100], Loss: -20.2660
Epoch [31/100], Loss: -19.9885
Epoch [32/100], Loss: -19.9500
Epoch [33/100], L

In [23]:
"""写成可以搜索的形式"""
def more_data(a, b):
    # 定义超参数
    input_size = dimensions
    hidden_dim = 10
    num_classes = 10      # MNIST有10个类别
    learning_rate = 0.01
    num_epochs = 100
    batch_size = 64
    #b = 1.5
    #a = 2

    # 实例化模型、定义损失函数和优化器
    model = SingleLayerNN(input_size, hidden_dim).to(device)
    criterion = FBMLoss(hidden_dim, 0.01, a, b)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    model.train()
    # 训练模型
    for epoch in range(num_epochs):
        for images, labels in train_loader:
            # 将图像和标签移动到 GPU 上
            images = images.view(-1, input_size).to(device)  # 展平图像并转移到 GPU
            labels = labels.to(device)  # 标签移动到 GPU
            #labels_one_hot = F.one_hot(labels, num_classes=num_classes).float()
            
            # 前向传播
            outputs = model(images)
            #loss = criterion(outputs, labels_one_hot, model.linear.weight)
            loss = criterion(outputs, labels, model.linear.weight)
            
            # 反向传播和优化
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        '''分析代码'''
        with torch.no_grad():
            local_data = train_features.view(-1, input_size).to(device)
            analyse_data = model(local_data).cpu()
            
            local_save_path = os.path.join(save_path, f"train2/hidden{hidden_dim}_a{a:.2f}_b{b:.2f}/")
            os.makedirs(local_save_path, exist_ok=True)

            visualize_2d(analyse_data, train_labels.cpu(), os.path.join(local_save_path, f"{epoch}"))   #画出最后的结果
            out_dis = fast_FBDistance(analyse_data, train_labels.cpu())
            # 计算对角项的和
            diagonal_sum = torch.diag(out_dis).sum()

            # 计算非对角项的和
            total_sum = out_dis.sum()
            non_diagonal_sum = total_sum - diagonal_sum
            with open(os.path.join(local_save_path, "distance.txt"), "a") as file:
                file.write("="*20+"\n"+f"epoch={epoch}"+"\n")
                file.write(str(out_dis) + "\n")  # 写入数据并换行
                F_B = non_diagonal_sum/diagonal_sum
                file.write(f"Bosen={diagonal_sum:.4f}\t Fermi={non_diagonal_sum:.4f}\t Fermi_Bosen={F_B:.4f}\n")
        
        #print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

for a in np.linspace(0.3, 2, 10):
    for b in np.linspace(0.3, 2, 10):
        more_data(a, b)

# 绘制在测试集上的表现

In [ ]:
with torch.no_grad():
    out_features = model(test_features.to(device)).cpu()
visualize_2d(out_features, test_labels)